In [4]:
import keras
from keras import layers
import tensorflow as tf
from keras import applications, models, layers

import os
print(os.listdir('./data'))


train_dataset = keras.utils.image_dataset_from_directory(
    './data',                       
    labels='inferred',             
    label_mode='int',              
    batch_size=32,                 
    image_size=(256, 256),         
    shuffle=True,                  
    validation_split=0.5,          
    subset='training',             
    seed=123                       
)

validation_dataset = keras.utils.image_dataset_from_directory(
    './data',                    
    labels='inferred',           
    label_mode='int',            
    batch_size=32,               
    image_size=(256, 256),       
    shuffle=True,                
    validation_split=0.2,        
    subset='validation',         
    seed=123                     
)

# normalize pixel values to [0, 1]
normalization_layer = layers.Rescaling(1./255)

train_dataset = train_dataset.map(lambda x, y: (normalization_layer(x), y))
validation_dataset = validation_dataset.map(lambda x, y: (normalization_layer(x), y))

# Prefetch to optimize performance during training
# AUTOTUNE = tf.data.AUTOTUNE
# train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
# validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)

base_model = applications.MobileNetV2(input_shape=(256, 256, 3), include_top=False, weights='imagenet')
base_model.trainable = False  

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Assuming binary classification (fire risk vs no fire risk)
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# training
history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=10
)

# unfreeze base model layers and fine tune
base_model.trainable = True
for layer in base_model.layers[:-40]:
    layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='binary_crossentropy',
              metrics=['accuracy'])

fine_tune_history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=5
)

model.save('fire_risk_model.h5')

['fire', 'nofire']
Found 4 files belonging to 2 classes.
Using 2 files for training.
Found 4 files belonging to 2 classes.
Using 0 files for validation.
Epoch 1/10


/var/folders/4s/0shkycv9613d_rw6h0lfrjqw0000gn/T/ipykernel_69681/2035116742.py:45: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = applications.MobileNetV2(input_shape=(256, 256, 3), include_top=False, weights='imagenet')


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.5000 - loss: 0.7946 - val_accuracy: 1.0000 - val_loss: 0.2076
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step - accuracy: 1.0000 - loss: 0.1619 - val_accuracy: 1.0000 - val_loss: 0.1157
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - accuracy: 1.0000 - loss: 0.0483 - val_accuracy: 1.0000 - val_loss: 0.0715
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - accuracy: 1.0000 - loss: 0.0177 - val_accuracy: 1.0000 - val_loss: 0.0456
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step - accuracy: 1.0000 - loss: 0.0074 - val_accuracy: 1.0000 - val_loss: 0.0308
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - accuracy: 1.0000 - loss: 0.0035 - val_accuracy: 1.0000 - val_loss: 0.0227
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step - accuracy: 1.0000 - loss: 0.0019 - val_accuracy: 1.0000 - val_loss: 0.0178
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - accuracy: 1.0000 - loss: 0.0012 - val_accuracy: 1.0000 - val_loss: 0.0146
Epoch 9/10
1

In [5]:
train_loss, train_accuracy = model.evaluate(train_dataset)
print(f'Training Loss: {train_loss}')
print(f'Training Accuracy: {train_accuracy}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 1.0000 - loss: 2.9475e-04
Training Loss: 0.0002947513712570071
Training Accuracy: 1.0
